# Demonstration of AIPW with a single sample
Data generating mechanism comes from Funk et al. AJE (2011). See online supplementary information for details (or the `dgm.py` file)

In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Loading the data

In [2]:
df = pd.read_csv("dr_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Z1    5000 non-null float64
Z2    5000 non-null float64
Z3    5000 non-null int64
X     5000 non-null int64
Y     5000 non-null float64
dtypes: float64(3), int64(2)
memory usage: 195.4 KB


The exposure is `X`, the outcome is `Y`, and `Z`'s indicate potential confounders. 

### Estimating Propensity Score (IPW)

In [3]:
f = sm.families.family.Binomial()
propensity_model = smf.glm("X ~ Z1 + Z3", df, family=f).fit()
propensity_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      X   No. Observations:                 5000
Model:                            GLM   Df Residuals:                     4997
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2992.3
Date:                Wed, 13 May 2020   Deviance:                       5984.6
Time:                        10:24:50   Pearson chi2:                 4.95e+03
No. Iterations:                     4   Covariance Type:             nonrobust
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7613      0.039     19.672      0.000       0.685       0.837
Z1            -0.7860      0.035    -22.160      0.000      -0.856      -0.716
Z3            -0.9347      0.067    -13.863      0.000      -1.067      -0.803
==============================================================================
"""

In [4]:
ps = propensity_model.predict(df)

### Estimating Outcome Model (g-formula)

In [5]:
outcome_model = smf.ols("Y ~ X + Z1 + Z3", df).fit()
outcome_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.220
Model:                            OLS   Adj. R-squared:                  0.219
Method:                 Least Squares   F-statistic:                     468.7
Date:                Wed, 13 May 2020   Prob (F-statistic):          2.45e-268
Time:                        10:24:50   Log-Likelihood:                -10554.
No. Observations:                5000   AIC:                         2.112e+04
Df Residuals:                    4996   BIC:                         2.114e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0969      0.053      1.819      0.069      -0.008       0.201
X             -0.0682      0.062     -1.097      0.273      -0.190       0.054
Z1             0.9565      0.030     31.682      0.000       0.897       1.016
Z3             1.0055      0.063     16.018      0.000       0.882       1.129
==============================================================================
Omnibus:                        1.810   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.405   Jarque-Bera (JB):                1.838
Skew:                           0.045   Prob(JB):                        0.399
Kurtosis:                       2.975   Cond. No.                         3.66
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [6]:
dfx = df.copy()
dfx["X"] = 1
y_x1 = outcome_model.predict(dfx)

dfx = df.copy()
dfx["X"] = 0
y_x0 = outcome_model.predict(dfx)

### Generating Pseudo-Outcomes

In [7]:
y_obs = np.asarray(df['Y'])
x_obs = np.asarray(df['X'])

y_x1_star = (y_obs*x_obs)/(ps) + (y_x1*(ps-x_obs))/(ps)
y_x0_star = (y_obs*(1-x_obs))/(1-ps) + (y_x0*(x_obs - ps))/(1-ps)

## AIPW Estimates

In [8]:
pseudo_y = y_x1_star - y_x0_star

# Point estimate:
ate = np.mean(pseudo_y)

# Variance:
var_ate = np.var(pseudo_y - ate, ddof=1) / df.shape[0]

# Confidence intervals:
ci_ate = (ate - 1.96*np.sqrt(var_ate), 
          ate + 1.96*np.sqrt(var_ate))

In [9]:
print(ate)
print(ci_ate)

-0.07010393035825216
(-0.1953023549707608, 0.05509449425425647)


For comparison, the g-formula and IPW estimates are provided below

In [10]:
# G-computation
np.mean(y_x1 - y_x0)

-0.06816575398930338

In [11]:
# IPW
np.mean((y_obs*x_obs)/ps - (y_obs*(1-x_obs))/(1-ps))

-0.1144067889068286